PACKAGE IMPORTS

In [1]:
# Pre Processing
import nltk
import pandas as pd 
import numpy as np
import re 
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

# Model building 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
import sklearn

# Model evaluation
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score


LOADING & CONVERTING CSV FILES TO PANDAS DATAFRAMES

In [2]:
df_test  = pd.read_csv('test_set.csv', skipinitialspace = True)
df_train = pd.read_csv('train_set.csv')

PRE - PROCESSING

Data Disrtibution 

In [3]:
# Chekcing the landguage Id data distribution
# All classes have equal distribution 
df_train['lang_id'].value_counts()

xho    3000
eng    3000
nso    3000
ven    3000
tsn    3000
nbl    3000
zul    3000
ssw    3000
tso    3000
sot    3000
afr    3000
Name: lang_id, dtype: int64

Data cleaning 

In [4]:
# Text cleaning function.

def CleanTweets(tweets):
    # Converts from upper case to lower case
    tweets = tweets.lower()
    # removing numbers
    tweets = re.sub(r'\d+','',tweets)
    # removes @ mentions 
    tweets = re.sub('@[\w]*','',tweets)
    # removes urls
    tweets = re.sub(r'https?:\/\/.*\/\w*','',tweets)
    # removes hashtags
    tweets = re.sub(r'#\w*','',tweets)
    # removes punctuation
    tweets = ''.join([l for l in tweets if l not in string.punctuation])   
    # Removes extra white space
    tweets = re.sub(r'\s\s+','',tweets)
    # removes the newline characters [\n] from pandas column
    tweets = tweets.replace('\n', ' ')
    # removes space infront of tweet
    tweets = tweets.strip()
    tweets = tweets.lstrip()
    return tweets
df_train['text'] = df_train['text'].apply(CleanTweets)
df_test['text']  = df_test['text'].apply(CleanTweets) 



FEATURE BUILDING

Language Id Conversion

In [5]:
# This function converts the responce variable lables to numerical values

def Language_to_num(df):
    # Allocate lang_id column to variable language    
    language = list(df['lang_id'])
    text_to_num = []  # Initialize open list assigend to variable text_to_num

    # Use for loop to assign numerical values to language Ids
    for i in language  :
        if i == 'xho'  :
            text_to_num.append(1)
        elif i == 'eng'    :
            text_to_num.append(2)
        elif i == 'nso'    :
            text_to_num.append(3)
        elif i == 'ven'    :
            text_to_num.append(4)
        elif i == 'tsn'    :
            text_to_num.append(5)
        elif i == 'nbl'    :
            text_to_num.append(6)
        elif i == 'zul'    :
            text_to_num.append(7)
        elif i == 'ssw'    :
            text_to_num.append(8)
        elif i == 'tso'    :
            text_to_num.append(9)
        elif i == 'sot'    :
            text_to_num.append(10)
        else :
            text_to_num.append(11)
            
        # Convert text_to_num from list to tuple
        def convert(list):
            return tuple(list)
        convert(text_to_num)
        
    # Assign values in lang_id column to new column language of df_test data frame
    df_train['language'] = df_train['lang_id']
    
    # Assign values in text_to_num list to new Id column of df_test data frame
    df['lang_id'] = text_to_num

    
    return df

# Initialize Language_to_num function to df_train data frame
df_train = Language_to_num(df_train)

MODEL BUILDING

Text Vectorization 

I TF-IDF Vectorizer [tfidf]

In [6]:
# Initialize TF-IDF vectorizer 
tfidf = TfidfVectorizer(stop_words='english', tokenizer= word_tokenize, analyzer='char', ngram_range=(1, 7)) 

# Fit and transform of train and test predictor feartures
vect_train_tfidf = tfidf.fit_transform(df_train['text'])
vect_test_tfidf  = tfidf.transform(df_test['text'])

II Count Vectorizer [cv]

In [7]:
# Initialize CountVectorizer 
cv = CountVectorizer(min_df = 2  , max_df = 0.5, ngram_range =(1,2))

# Fit and transform of train and test predictor feartures
vect_train_cv = cv.fit_transform(df_train['text'])
vect_test_cv  = cv.transform(df_test['text'])

TRAIN - TEST SPLIT

TF-IDF Vectorizer

In [8]:
# Train test splitting
X = vect_train_tfidf
Y = df_train['lang_id']

#Setting the train test ratio and assiging X,y(train, test) values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 42)

Count Vectorizer

In [9]:
# Train test splitting
X = vect_train_cv
Y = df_train['lang_id']

#Setting the train test ratio and assiging X,y(train, test) values
X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X, Y, test_size = 0.25, random_state = 42)

MODEL SELECTION

TF-IDF Vectorizer

In [12]:
# Logistic Classifier
# Classifier initialization, fit and prediction 
lc_tfidf = LogisticRegression(multi_class = 'ovr', solver = 'liblinear', random_state = 42) 
lc_tfidf.fit(X_train, y_train)
y_lc_tfidf = lc_tfidf.predict(X_test)

# Displaying model performance metrics
print(classification_report(y_test, y_lc_tfidf))
print(metrics.f1_score(y_test, y_lc_tfidf, average='macro'))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       761
           2       1.00      1.00      1.00       762
           3       1.00      1.00      1.00       782
           4       1.00      1.00      1.00       785
           5       1.00      1.00      1.00       746
           6       1.00      1.00      1.00       734
           7       1.00      0.99      1.00       735
           8       1.00      1.00      1.00       732
           9       1.00      1.00      1.00       700
          10       1.00      1.00      1.00       759
          11       1.00      1.00      1.00       754

    accuracy                           1.00      8250
   macro avg       1.00      1.00      1.00      8250
weighted avg       1.00      1.00      1.00      8250

0.998660650859696


In [ ]:
# Support Vector Classifier [svc]
# Classifier initialization, fit and prediction
svc_tfidf = SVC(C = 10, gamma = 0.01)
svc_tfidf.fit(X_train, y_train)
y_svc_tfidf_pred = svc_tfidf.predict(X_test)

# Displaying model performance metrics
print(classification_report(y_test, y_svc_tfidf_pred))
print(metrics.f1_score(y_test, y_svc_tfidf_pred, average='macro'))

In [11]:
# Multinomial Classifier Tweaked [mt] (Best Performing Model)
# Classifier initialization, fit and prediction
mt_tfidf = MultinomialNB(alpha = 0.002 , fit_prior = False )
mt_tfidf.fit(X_train, y_train)
y_mt_tfidf = mt_tfidf.predict(X_test)

# Displaying model performance metrics
print(classification_report(y_test, y_mt_tfidf))
print(metrics.f1_score(y_test, y_mt_tfidf, average='macro'))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       761
           2       1.00      1.00      1.00       762
           3       1.00      1.00      1.00       782
           4       1.00      1.00      1.00       785
           5       1.00      1.00      1.00       746
           6       1.00      1.00      1.00       734
           7       1.00      1.00      1.00       735
           8       1.00      1.00      1.00       732
           9       1.00      1.00      1.00       700
          10       1.00      1.00      1.00       759
          11       1.00      1.00      1.00       754

    accuracy                           1.00      8250
   macro avg       1.00      1.00      1.00      8250
weighted avg       1.00      1.00      1.00      8250

0.9996392021548123


In [ ]:
# SDG Classifier [sgd]
# Classifier initialization, fit and prediction
sgd_tfidf = SGDClassifier(loss = "hinge", penalty = "l2", max_iter=10) 
sgd_tfidf.fit(X_train, y_train)
y_sgd_tfidf = sgd_tfidf.predict(X_test)

# Displaying model performance metrics
print(classification_report(y_test, y_sgd_tfidf))
print(metrics.f1_score(y_test, y_sgd_tfidf, average='macro'))

In [ ]:
# Linear SVC [lsvc]
# Classifier initialization, fit and prediction
lsvc_tfidf = LinearSVC()
lsvc_tfidf.fit(X_train, y_train)
y_lsvc_tfidf = lsvc_tfidf.predict(X_test)

# Displaying model performance metrics
print(classification_report(y_test, y_lsvc_tfidf))
print(metrics.f1_score(y_test, y_lsvc_tfidf, average='macro'))

In [ ]:
# Multinomial Standard [ms]
# Classifier initialization, fit and prediction
ms_tfidf = MultinomialNB()
ms_tfidf.fit(X_train, y_train)
y_ms_tfidf = ms_tfidf.predict(X_test)

# Displaying model performance metrics
print(classification_report(y_test, y_ms_tfidf))
print(metrics.f1_score(y_test, y_ms_tfidf, average='macro'))

Count Vectorizer

In [ ]:
# Logistic Classifier [lc]
# Classifier initialization, fit and prediction
lc_cv = LogisticRegression(multi_class = 'ovr', solver = 'liblinear', random_state = 42) 
lc_cv.fit(X_train_cv, y_train_cv)
y_lc_cv = lc_cv.predict(X_test_cv)

# Displaying model performance metrics
print(classification_report(y_test_cv, y_lc_cv))
print(metrics.f1_score(y_test, y_lc_cv, average='macro'))

In [ ]:
# Support Vector Classifier [svc]  
# Classifier initialization, fit and prediction
svc_cv = SVC(C = 10, gamma = 0.01)
svc_cv.fit(X_train_cv, y_train_cv)
y_svc_cv_pred = svc_cv.predict(X_test_cv)

# Displaying model performance metrics
print(classification_report(y_test_cv, y_svc_cv_pred))
print(metrics.f1_score(y_test, y_svc_cv_pred, average='macro'))

In [ ]:
# Miltinomial Tweeked [mt]
# Classifier initialization, fit and prediction
mt_cv = MultinomialNB(fit_prior=False)
mt_cv.fit(X_train_cv, y_train_cv)
y_mt_cv = mt_cv.predict(X_test_cv)

# Displaying model performance metrics
print(classification_report(y_test_cv, y_mt_cv))
print(metrics.f1_score(y_test, y_mt_cv, average='macro'))

In [ ]:
# SGD Classifier [sgd]
# Classifier initialization, fit and prediction
sgd_cv = SGDClassifier(loss="hinge", penalty="l2", max_iter=10) 
sgd_cv.fit(X_train_cv, y_train_cv)
y_sgd_cv = sgd_cv.predict(X_test_cv)

# Displaying model performance metrics
print(classification_report(y_test_cv, y_sgd_cv))
print(metrics.f1_score(y_test, y_sgd_cv, average='macro'))

In [ ]:
# Linear SVC [lsvc]
# Classifier initialization, fit and prediction
lsvc_cv = LinearSVC()
lsvc_cv.fit(X_train_cv, y_train_cv)
y_lsvc_cv = lsvc_cv.predict(X_test_cv)

# Displaying model performance metrics
print(classification_report(y_test_cv, y_lsvc_cv))
print(metrics.f1_score(y_test, y_lsvc_cv, average='macro'))

In [ ]:
# Multinomial Standard [ms]
# Classifier initialization, fit and prediction
ms_cv = MultinomialNB()
ms_cv.fit(X_train_cv, y_train_cv)
y_ms_cv = ms_cv.predict(X_test_cv)

# Displaying model performance metrics
print(classification_report(y_test_cv, y_ms_cv))
print(metrics.f1_score(y_test, y_ms_cv, average='macro'))

Model Stacking

In [ ]:
# Stack Classifier initialization and parameter setting
estimators = [('lc_cv', lc_cv)]
stack = StackingClassifier(estimators = estimators,
                           final_estimator = LogisticRegression(C=20, class_weight='balanced', max_iter=10000))

In [ ]:
# Stack Classifier (Train, Test) fitting and ensemble model prediction
stack.fit(X_train, y_train)
y_pred_lc_cv = stack.predict(vect_test_cv)
print(metrics.classification_report(y_test, y_pred_lc_cv))

KAGGLE SUBMISSION

Sample Refference

In [ ]:
# Checking sample submission format 
submission = pd.read_csv('sample_submission.csv')
submission.head()

Model Prediction

In [ ]:
# Checking the shape of [Count Vectorizer] vectorized train and test predictor features
vect_train_cv.shape, vect_test_cv.shape

In [ ]:
# Checking the shape of [TF-IDF Vectorizer] vectorized train and test predictor features
vect_train_tfidf.shape, vect_test_tfidf.shape

In [ ]:
# Predicting the Test Dataset predictor features
y_pred_mt_tfidf = mt_tfidf.predict(vect_test_tfidf)

Feature Building

In [ ]:
'''
This function converts the responce feature predictions from
numerical values to text values for anlaysis by the Kaggle auto-grader.
'''

def Num_to_language(var):    
    # Initialize open list assigend to variable text_to_num
    num = list(var) 

    # Allocate lang_id column to variable language    
    language = []

    # Use for loop to assign numerical values to language Ids
    for i in num  :
        
        if i == 1  :
            language.append('xho')
        elif i == 2    :
            language.append('eng')
        elif i == 3    :
            language.append('nso')
        elif i == 4    :
            language.append('ven')
        elif i == 5    :
            language.append('tsn')
        elif i == 6    :
            language.append('nbl')
        elif i == 7    :
            language.append('zul')
        elif i == 8    :
            language.append('ssw')
        elif i == 9    :
            language.append('tso')
        elif i == 10    :
            language.append('sot')
        else :
            language.append('afr')
        
    df_test['lang_id'] = language
    return 


In [ ]:
# Converting the predictions from the numbers to characters
Num_to_language(y_pred_mt_tfidf)

Building Submission File

In [ ]:
# Assigning df_test index column values to index variable
index = df_test['index']
# Assigning df_test lang_id column values to lang_id variable
lang_id = df_test['lang_id']

In [ ]:
# Combining index and prediction values into pandas dataframe and assigning them to a variable 
submission_mt_tfidf = pd.DataFrame({'index' : index, 'lang_id' : lang_id})

In [ ]:
# Converting submission file from dataframe to csv file format and downloading file to the directory
submission_mt_tfidf.to_csv("Submission_mt_tfidf_v4.csv", index = False)